In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from selenium import webdriver
driver = webdriver.Firefox()

base_url = 'https://www.espncricinfo.com/records/tournament/averages-bowling/icc-cricket-world-cup-2023-24-15338'
url = 'https://www.espncricinfo.com'

In [2]:
response = requests.get(base_url)
if response.status_code != 200:
    raise Exception('Failed to load page {}'.format(base_url))
doc = BeautifulSoup(response.text, 'html.parser')

player_urls =[]

table = doc.tbody

for row in table.find_all('tr', class_ = 'ds-table-row-compact-bottom'):
    column = row.find_all('td')
    player = column[0].span.a
    player_urls.append(url + player['href'])   

In [3]:
player_urls[:5]

['https://www.espncricinfo.com/cricketers/sean-abbott-398666',
 'https://www.espncricinfo.com/cricketers/abdullah-shafique-922941',
 'https://www.espncricinfo.com/cricketers/colin-ackermann-379774',
 'https://www.espncricinfo.com/cricketers/agha-salman-623977',
 'https://www.espncricinfo.com/cricketers/moeen-ali-8917']

name, team, image, battingStyle, bowlingStyle, playingRole, description

In [4]:
wc_player_data = {'name' : [], 'team': [], 'image': [], 'playingRole': [], 'battingStyle': [], 'bowlingStyle': [], 'description': []}

In [8]:
def getPlayerImage(url, player):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html')
    return soup.find('img', alt=player)['src']

In [9]:
def get_players_data():
    for player in player_urls:
        r = requests.get(player)
        if r.status_code != 200:
            raise Exception('Failed to load page {}'.format(player))
        soup = BeautifulSoup(r.text, 'html.parser')

        name = soup.find('div', class_='ds-flex ds-space-x-5').find('h1', class_='ds-text-title-l ds-font-bold').text
        team = soup.find('div', class_='ds-flex ds-space-x-5').span.text
        
        image = getPlayerImage(player, name)
        
        playingRole = soup.find('div', class_='ds-flex ds-space-x-5').find_all('span')[2].text
        
        player_details = soup.find_all('span', class_ = 'ds-text-title-s ds-font-bold ds-text-typo')
        
        if(soup.find_all('p', class_ = 'ds-text-tight-m ds-font-regular ds-uppercase ds-text-typo-mid3')[3].text != "Nicknames"):
            battingStyle = player_details[3].text
            bowlingStyle = player_details[4].text
        else:
            battingStyle = player_details[4].text
            bowlingStyle = player_details[5].text
            
        if(soup.find('div', class_ = 'ci-player-bio-content') is not None):
            all_p_tags = soup.find('div', class_ = 'ci-player-bio-content').find_all('p')[:2]
            description = soup.find('div', class_ = 'ci-player-bio-content').find_all('p')[0].text
        else:
            description = ''
            
        wc_player_data['name'].append(name)
        wc_player_data['team'].append(team)
        wc_player_data['image'].append(image)
        wc_player_data['playingRole'].append(playingRole)
        wc_player_data['battingStyle'].append(battingStyle)
        wc_player_data['bowlingStyle'].append(bowlingStyle)
        wc_player_data['description'].append(description)
        
        print(name, " data extracted")

    return pd.DataFrame(wc_player_data)

In [10]:
players_data = get_players_data()

Sean Abbott  data extracted
Abdullah Shafique  data extracted
Colin Ackermann  data extracted
Agha Salman  data extracted
Moeen Ali  data extracted
Charith Asalanka  data extracted
Ravichandran Ashwin  data extracted
Gus Atkinson  data extracted
Azmatullah Omarzai  data extracted
Babar Azam  data extracted
Jonny Bairstow  data extracted
Wesley Barresi  data extracted
Temba Bavuma  data extracted
Trent Boult  data extracted
Harry Brook  data extracted
Jasprit Bumrah  data extracted
Jos Buttler  data extracted
Alex Carey  data extracted
Dushmantha Chameera  data extracted
Mark Chapman  data extracted
Gerald Coetzee  data extracted
Devon Conway  data extracted
Pat Cummins  data extracted
Sam Curran  data extracted
Quinton de Kock  data extracted
Bas de Leede  data extracted
Dhananjaya de Silva  data extracted
Aryan Dutt  data extracted
Scott Edwards  data extracted
Sybrand Engelbrecht  data extracted
Fakhar Zaman  data extracted
Fazalhaq Farooqi  data extracted
Lockie Ferguson  data extra

In [11]:
players_data.head(20)

,name,team,image,playingRole,battingStyle,bowlingStyle,description
0,Sean Abbott,Australia,"https://img1.hscicdn.com/image/upload/f_auto,t...",Bowling Allrounder,Right hand Bat,Right arm Fast medium,Sean Abbott has developed into one of the most...
1,Abdullah Shafique,Pakistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Top order Batter,Right hand Bat,Right arm Offbreak,
2,Colin Ackermann,Netherlands,"https://img1.hscicdn.com/image/upload/f_auto,t...",Batting Allrounder,Right hand Bat,Right arm Offbreak,Having relinquished his South African qualific...
3,Agha Salman,Pakistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Allrounder,Right hand Bat,Right arm Offbreak,
4,Moeen Ali,England,"https://img1.hscicdn.com/image/upload/f_auto,t...",Batting Allrounder,Left hand Bat,Right arm Offbreak,Moeen Ali was tipped for the top long before h...
5,Charith Asalanka,Sri Lanka,"https://img1.hscicdn.com/image/upload/f_auto,t...",Batting Allrounder,Left hand Bat,Right arm Offbreak,
6,Ravichandran Ashwin,India,https://wassets.hscicdn.com/static/images/lazy...,Bowling Allrounder,Right hand Bat,Right arm Offbreak,R Ashwin took the tricks and skills he learned...
7,Gus Atkinson,England,"https://img1.hscicdn.com/image/upload/f_auto,t...",Bowler,Right hand Bat,Right arm Fast medium,It may seem odd to talk of 2022 as a breakthro...
8,Azmatullah Omarzai,Afghanistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Allrounder,Right hand Bat,Right arm Medium fast,
9,Babar Azam,Pakistan,"https://img1.hscicdn.com/image/upload/f_auto,t...",Batter,Right hand Bat,Right arm Offbreak,"A right-hand, top-order batsman known for his ..."


In [12]:
len(players_data)

151

In [13]:
players_data.to_csv('./data/wc2023_players.csv', index = None)